Plan :

1 - Acquisition des donnees

2 - API => S3 

3 - S3 => MongoDB 

4 - MongoBD => SQL 

5 - A faire/résoudre

Regroupement des opérations de même type (script d'utilisation des API, script de création des BDD (SQL et NoSQL), scripts connexion et d'd'insertion des données dans les BDD)

Organisation des différents scripts dans une structure de fichiers claires sur Github

Création d'un notebook d'explication et de démonstration des utilisations des différents scripts 

1 - Acquisition des donnees 

    1.1 Les sources de donnees
        Nous avons decides de diversifier nos sources de donnees pour recolter un maximum d'infomrations pour  fournir notre base de donnees (quitte a en filter plus loin dans le pipeline)
        - Lufthansa OPEN API 
            La lufthansa OPEN API nous permet essentiellement de recuperer les donnees statiques et variables (dont nous verrons des exemples plus bas quand nous testerons les API correspondants) 
        - airlabs API 
            L'avantage de l'API airlabs est qu'elle nous fourni en donnees temps reel, nottamment sur les vols en cours
    1.2 Organisation du code source
    
    ├── 0_consignes
    │   └── Fiche Projet DST Airlines.pdf
    ├── 1_data_source_acquisition
    │   └── Etape1.ipynb
    ├── README.md
    └── src
        ├── dst_nosql
        │   ├── download_file_from_s3.py
        │   ├── mongodb.py
        │   ├── purge_temp_folder.py
        │   ├── read_file_from_s3.py
        │   └── store_json_S3.py
        ├── dst_source
        │   ├── AirlabsStatic.py
        │   ├── AirlabsVariable.py
        │   ├── DstRealtime.py
        │   ├── LufthansaStatic.py
        │   ├── LufthansaVariable.py
        │   ├── dst_utils.py
        │   └── test.py
        ├── dst_sql
        │   ├── create_sqlbdd_and_table.py
        │   ├── inject_data.py
        │   └── sql_queries.py
        └── rendu_notebook.ipynb

    Dans la branche main de notre repo git, au sein du repertoire src, qui contient comme son nom l'indique tout notre code source reparti selon 3 repertoires : 
    - le repertoire "dst_nosql" va contenir le code se rapportant au stockage de donnees sur s3 et sur mongoDB
    - le repertoire "dst_sql" va contenir tout le code qui nous permet la creation, l'injection et les requete de notre base de donnees relationnelle 
    - le repertoire "dst_source" va quand a lui contient le code python qui implemente notre api qui permet de requeter nos deux sources de donnees efficacement (detaillee dans 1.3)

    1.3 Recuperation des donnees : Ici nous allons detailler et donner des exemples d'utilisation de notre API

LufthansaStatic : Cette classe contient les methodes peremettant la recuperation des donnees statiques lufthansa (villes, pays, aeroports...)

In [3]:
import dst_utils as lu
import LufthansaStatic 
import LufthansaVariable 
import AirlabsStatic
import AirlabsVariable
import DstRealtime

import requests


Pour pouvoir utiliser la lufthansa OPEN API,  il faut tout d'abord nous identifier aupres du endpoint d'authentification. Pour ce faire, nous avons implemente une classe Authentification (dans le module dst_utils)
Authentification

In [12]:
auth = lu.Authentication(client_key = "bnagmhp63tmy78hfpa3v2bgw", client_secret = "MVZHkTAMqc4y4Esz2ye5")

Ceci nous permet de recuprer un header qui nous permet de requeter l'API

In [13]:
header = auth.get_header()

Ensuite il faut instancier la classe `RequestFactory` qui elle implemente la methode de creation de requete a l'aide du header sus recupere

In [14]:
rf = lu.RequestFactory(header)

Nous pouvons maintenant instancier nos classes `LufthansaVariable` et `LufthansaStatic` dont les constructeurs prennent notre factory qui sera utilise aux differents appels au methodes de ces classes 

In [15]:
lufstat = LufthansaStatic.LufthansaStatic(rf)
lufvar  = LufthansaVariable.LufthansaVariable(rf)

Exemples d'appels aus deux classes

In [9]:
#affichage des donnees du vol en provenance de New York et a destination de France en date du 01/08/2022
lufvar.get_flight_route("JFK", "FRA",  "2022-08-01")

https://api.lufthansa.com/v1/operations/flightstatus/route/JFK/FRA/2022-08-01::200
Writing file : /Users/houda.el-mir/DST-airlines/src/dst_source/tmp_json/get_flight_route_lufJFK_FRA_2022-08-01.json


{'FlightStatusResource': {'Flights': {'Flight': [{'Departure': {'AirportCode': 'JFK',
      'ScheduledTimeLocal': {'DateTime': '2022-08-01T16:05'},
      'ScheduledTimeUTC': {'DateTime': '2022-08-01T20:05Z'},
      'ActualTimeLocal': {'DateTime': '2022-08-01T16:20'},
      'ActualTimeUTC': {'DateTime': '2022-08-01T20:20Z'},
      'TimeStatus': {'Code': 'OT', 'Definition': 'Flight On Time'},
      'Terminal': {'Name': '1', 'Gate': '005'}},
     'Arrival': {'AirportCode': 'FRA',
      'ScheduledTimeLocal': {'DateTime': '2022-08-02T05:45'},
      'ScheduledTimeUTC': {'DateTime': '2022-08-02T03:45Z'},
      'ActualTimeLocal': {'DateTime': '2022-08-02T05:26'},
      'ActualTimeUTC': {'DateTime': '2022-08-02T03:26Z'},
      'TimeStatus': {'Code': 'FE', 'Definition': 'Flight Early'},
      'Terminal': {'Name': '1', 'Gate': 'Z62'}},
     'MarketingCarrier': {'AirlineID': 'LH', 'FlightNumber': '401'},
     'OperatingCarrier': {'AirlineID': 'LH', 'FlightNumber': '401'},
     'Equipment': {'Aircr

In [10]:
#affichage des vols (plusieurs vols a la fois pour une periode) du 01 au 29 juillet 2022 
lufvar.get_flights(write_json=False,startDate="01JUL22",endDate="29JUL22",daysOfOperation="1234567", timeMode = "UTC", flightType="passenger")

https://api.lufthansa.com/v1/flight-schedules/flightschedules/passenger?airlines=LH&startDate=01JUL22&endDate=29JUL22&daysOfOperation=1234567&timeMode=UTC::206


[{'airline': 'LH',
  'flightNumber': 1,
  'suffix': '',
  'periodOfOperationUTC': {'startDate': '01JUL22',
   'endDate': '01JUL22',
   'daysOfOperation': '    5  '},
  'periodOfOperationLT': {'startDate': '01JUL22',
   'endDate': '01JUL22',
   'daysOfOperation': '    5  '},
  'legs': [{'sequenceNumber': 1,
    'origin': 'HAM',
    'destination': 'FRA',
    'serviceType': 'J',
    'aircraftOwner': 'LH',
    'aircraftType': '321',
    'aircraftConfigurationVersion': 'C200',
    'registration': '',
    'op': True,
    'aircraftDepartureTimeUTC': 240,
    'aircraftDepartureTimeDateDiffUTC': 0,
    'aircraftDepartureTimeLT': 360,
    'aircraftDepartureTimeDateDiffLT': 0,
    'aircraftDepartureTimeVariation': 120,
    'aircraftArrivalTimeUTC': 310,
    'aircraftArrivalTimeDateDiffUTC': 0,
    'aircraftArrivalTimeLT': 430,
    'aircraftArrivalTimeDateDiffLT': 0,
    'aircraftArrivalTimeVariation': 120}],
  'dataElements': [{'startLegSequenceNumber': 1,
    'endLegSequenceNumber': 1,
    'id':

Pour ce qui est de la classe LufthansaStatic, il important de noter que les methodes de recuperation en "bulk" vont faire plusieurs requetes (grace a une methode privee de la classe) pour recurperer toutes les donnees. La lufthansa OPEN API nous oblige a faire ainsi car tres limitante sur le nombre d'elements dans la response (100 a la fois au maximum) 

In [20]:
# Recuperation des donnees de tous les aerports
lufstat.get_cities_data_luf()


https://api.lufthansa.com/v1/mds-references/cities/?lang=EN&limit=100&offset=0::200
https://api.lufthansa.com/v1/mds-references/cities/?lang=EN&limit=100&offset=100::200
https://api.lufthansa.com/v1/mds-references/cities/?lang=EN&limit=100&offset=200::200
https://api.lufthansa.com/v1/mds-references/cities/?lang=EN&limit=100&offset=300::200
https://api.lufthansa.com/v1/mds-references/cities/?lang=EN&limit=100&offset=400::200
https://api.lufthansa.com/v1/mds-references/cities/?lang=EN&limit=100&offset=500::200
https://api.lufthansa.com/v1/mds-references/cities/?lang=EN&limit=100&offset=600::200
https://api.lufthansa.com/v1/mds-references/cities/?lang=EN&limit=100&offset=700::200
https://api.lufthansa.com/v1/mds-references/cities/?lang=EN&limit=100&offset=800::200
https://api.lufthansa.com/v1/mds-references/cities/?lang=EN&limit=100&offset=900::200
https://api.lufthansa.com/v1/mds-references/cities/?lang=EN&limit=100&offset=1000::200
https://api.lufthansa.com/v1/mds-references/cities/?lan

https://api.lufthansa.com/v1/mds-references/cities/?lang=EN&limit=100&offset=9500::200
https://api.lufthansa.com/v1/mds-references/cities/?lang=EN&limit=100&offset=9600::200
https://api.lufthansa.com/v1/mds-references/cities/?lang=EN&limit=100&offset=9700::200
https://api.lufthansa.com/v1/mds-references/cities/?lang=EN&limit=100&offset=9800::200
https://api.lufthansa.com/v1/mds-references/cities/?lang=EN&limit=100&offset=9900::200
https://api.lufthansa.com/v1/mds-references/cities/?lang=EN&limit=100&offset=10000::200
https://api.lufthansa.com/v1/mds-references/cities/?lang=EN&limit=100&offset=10100::200
https://api.lufthansa.com/v1/mds-references/cities/?lang=EN&limit=100&offset=10200::200
https://api.lufthansa.com/v1/mds-references/cities/?lang=EN&limit=100&offset=10300::200
https://api.lufthansa.com/v1/mds-references/cities/?lang=EN&limit=100&offset=10400::200
https://api.lufthansa.com/v1/mds-references/cities/?lang=EN&limit=100&offset=10500::200
https://api.lufthansa.com/v1/mds-refe

{'Cities': {'City': [{'CityCode': 'ZTV',
    'CountryCode': 'US',
    'Names': {'Name': {'@LanguageCode': 'EN', '$': 'Sturtevant'}},
    'UtcOffset': '-06:00',
    'TimeZoneId': 'America/Chicago',
    'Airports': {'AirportCode': 'ZTV'}},
   {'CityCode': 'ZTW',
    'CountryCode': 'CN',
    'Names': {'Name': {'@LanguageCode': 'EN', '$': 'Tsuen Wan'}},
    'UtcOffset': '+08:00',
    'TimeZoneId': 'Asia/Shanghai',
    'Airports': {'AirportCode': 'ZTW'}},
   {'CityCode': 'ZTX',
    'CountryCode': 'CN',
    'Names': {'Name': {'@LanguageCode': 'EN', '$': 'Tsuen Wan'}},
    'UtcOffset': '+08:00',
    'TimeZoneId': 'Asia/Shanghai',
    'Airports': {'AirportCode': 'ZTX'}},
   {'CityCode': 'ZTZ',
    'CountryCode': 'DE',
    'Names': {'Name': {'@LanguageCode': 'EN', '$': 'Chemnitz'}},
    'UtcOffset': '+01:00',
    'TimeZoneId': 'Europe/Berlin',
    'Airports': {'AirportCode': 'ZTZ'}},
   {'CityCode': 'ZUC',
    'CountryCode': 'CA',
    'Names': {'Name': {'@LanguageCode': 'EN', '$': 'Ignace'}},
 

#TODO: Finir redaction 

4 - MongoDB => SQL

L'ocjectif de cette étape est de faire passer les données qui sont sous le format json/mongodb à une BDD sql. Pourquoi ?
- Les màj des vols fréquentes s'apparentent à des transactions. Il est plus judicieux d'utiliser SQL lorsqu'il s'agit de transactions.
- La BDD mongodb sera davantage utilisée pour faire des transform sur les documents json. 
- Pour intégrer les données sur dash, il est plus simple d'avoir nos données sous format dataframe. Les dataframes seront plus simples à créer à partir de table sql qui contiennent déjà du lien entre elles. Le dataframe sera directement utilisable sur dash après une simple conversion de la requête sql en dataframe.
- Notre objectif final sera d'afficher simplement les informations d'un vol en cours ou non. Un objcectif d'analyse des données n'est pas encore envisagé mais reste toutefois une possibilité.

In [5]:
! python create_sqlbdd_and_table.py

2022-08-05 14:35:38,117 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-08-05 14:35:38,117 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("airports")
2022-08-05 14:35:38,117 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-05 14:35:38,117 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("airports")
2022-08-05 14:35:38,117 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-05 14:35:38,117 INFO sqlalchemy.engine.Engine 
CREATE TABLE airports (
	iata_code VARCHAR NOT NULL, 
	country_code VARCHAR, 
	icao_code VARCHAR, 
	lat FLOAT, 
	lng FLOAT, 
	name VARCHAR, 
	PRIMARY KEY (iata_code)
)


2022-08-05 14:35:38,117 INFO sqlalchemy.engine.Engine [no key 0.00005s] ()
2022-08-05 14:35:38,118 INFO sqlalchemy.engine.Engine COMMIT
2022-08-05 14:35:38,119 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-08-05 14:35:38,119 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("airports")
2022-08-05 14:35:38,119 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-05 14:35:38,119 INFO 

Le script create_sqlbdd_and_table.py permet de créer la base de données sql nommée "dst-airlines.db" puis de créer les différentes tables qui composeront cette BDD. A savoir :
- Une table airports (iata_code PK)
- Une table countries (code PK)
- Une table cities (city_code PK / country_code FK de la table countries)
- Une table aircraft (hex PK)
- Une table real_time_flights (flight_number PK / dep_iata FK de la table airports / arr_iata FK de la table airports)
- Une table delays_flights (id autoincrementé PK / flight_number_date FK de la table real_time_flights / dep_iata FK de la table airports / arr_iata FK de la table airports)
- Une table test (uniquement pour faire des tests)





In [6]:
! python inject_data.py

Le fichier inject_data.py permet d'automatiser l'injection les données issues de mongodb dans les tables de la BDD sql créée précédemment (fichier create_sqlbdd_and_table.py).
Les principales étapes du script inject_data.py sont les suivantes :
- Importer dans le fichier la BDD mongodb qui  sont stockées sur le cloud
- Importer les différentes collections de la BDD mongodb
- Importer la BDD sql qui a précédemment été créée et qui se nomme "dst_airlines.db"
- Automatiser l'injection de données pour chaque tables

Note : Nous avons un problème de valeur manquantes pour automatiser la création de toutes nos tables (c'est un problème que nous n'avons pas encore réussi à résoudre). Dans ce notebook, nous allons montrer uniquement l'automatisation de création d'une table "test" à partir de champs non manquants. 

In [7]:
! python sql_queries.py

[('0', 7840, 'en-route'), ('1', 2209, 'en-route'), ('2', 10675, 'en-route'), ('3', 10066, 'en-route'), ('4', 12192, 'en-route'), ('5', 10058, 'en-route'), ('6', 9753, 'en-route'), ('7', 10058, 'en-route'), ('8', 10965, 'en-route'), ('9', 7597, 'en-route'), ('10', 3680, 'en-route'), ('11', 11277, 'en-route'), ('12', 9448, 'en-route'), ('13', 11887, 'en-route'), ('14', 10972, 'en-route'), ('15', 11582, 'en-route'), ('16', 10668, 'en-route'), ('17', 10363, 'en-route'), ('18', 10363, 'en-route'), ('19', 12192, 'en-route'), ('20', 10972, 'en-route'), ('21', 10972, 'en-route'), ('22', 10668, 'en-route'), ('23', 10668, 'en-route'), ('24', 10363, 'en-route'), ('25', 10370, 'en-route'), ('26', 10363, 'en-route'), ('27', 10073, 'en-route'), ('28', 10066, 'en-route'), ('29', 9761, 'en-route'), ('30', 9753, 'en-route'), ('31', 7924, 'en-route'), ('32', 6705, 'en-route'), ('33', 5844, 'en-route'), ('34', 0, 'en-route'), ('35', 10972, 'en-route'), ('36', 10668, 'en-route'), ('37', 10363, 'en-route')

Le script sql_queries.py permet de faire des requêtes sql.
En l'occurence, la requête suivante a été effectuées :
"SELECT * FROM test;"

La table test est celle dont les données ont été injectées automatiquement depuis la BDD mongodb. Cela prouve qu'il est possible d'automatiser l'injection des données pour toutes nos tables. Il faut juste que l'on arrive à gérer les valeurs manquantes afin de créer automatiquement toutes nos tables. 

5 - A faire/résoudre

Partie mongobd => sql :
- gérer données manquantes dans les json
- ajout autimatique de la date du jour de la requête dans la table real_time_flight
- voir pour la mise à jour des champs dans la BDD sql
- uen fois qu'on aura toutes nos tables sql : tester la cohérence du lien entre nos table avec des jointures